In [ ]:
! pip install pandas
!pip install numpy
!pip install scikit-survival
!pip install tqdm

In [ ]:
import pandas as pd
hospit_non_chr=pd.read_csv("base_hospit_non_chr_anonyme.csv")
hospit_full_hospit=pd.read_csv("base_full_hospit_anonyme.csv")
consommant=pd.read_csv("consommant_anonyme.csv")
patient=pd.read_csv("patient_anonyme.csv")

In [ ]:
hospit_non_chr

In [ ]:
patient

In [ ]:
patient["date_h0"]=pd.to_datetime(patient["date_h0"])
patient["dte_deces"]=pd.to_datetime(patient["dte_deces"])

In [ ]:
patient["Nb_survie"]=patient["dte_deces"]-patient["date_h0"]
patient["Mort"]= [x.days>0 for x in patient["Nb_survie"]]
#patient=patient.drop(columns=["Survie"])

In [ ]:
patient["Nb_survie"]=patient["Nb_survie"].fillna(pd.Timedelta(3500, "d"))

In [ ]:
patient

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from sksurv.nonparametric import kaplan_meier_estimator

time, survival_prob = kaplan_meier_estimator(patient["Mort"], patient["Nb_survie"])
days=time.astype('timedelta64[D]')
days/np.timedelta64(1,'D')
plt.step(days, survival_prob, where="post")
plt.ylabel("Probabilité de survie $\hat{S}(t)$")
plt.xlabel("Temps en jours $t$")

In [ ]:
patient

In [ ]:
hospit_full_hospit

In [ ]:
from tqdm import tqdm

patient_mort=patient.loc[patient["Mort"]]
hospit_mort = []
hospit_nb_survie=[]
code_patient_mort=np.array(patient_mort.CODE_PATIENT_ENC)
for i in tqdm(range(len(hospit_full_hospit.BEN_NIR_IDT_ENC))):
    x=hospit_full_hospit.BEN_NIR_IDT_ENC.iloc[i]
    hospit_nb_survie.append(patient[patient["CODE_PATIENT_ENC"]==x]["Nb_survie"].iloc[0])
    if x in code_patient_mort:
        hospit_mort.append(True)
    else:
        hospit_mort.append(False)
hospit_full_hospit["Mort"]=hospit_mort
hospit_full_hospit["Nb_survie"]=hospit_nb_survie

In [ ]:
hospit_full_hospit

In [ ]:
patient["Mort"].sum()

In [ ]:
hospit_full_hospit["Mort"].sum()

In [ ]:
hospit_nb_survie[0]

In [ ]:
for treatment_type in (0, 1):
    mask_treat = hospit_full_hospit["CHOC"] == treatment_type
    time_treatment, survival_prob_treatment = kaplan_meier_estimator(
        hospit_full_hospit["Mort"][mask_treat],
        hospit_full_hospit["Nb_survie"][mask_treat])
    days=time_treatment.astype('timedelta64[D]')
    days/np.timedelta64(1,'D')
    plt.step(days, survival_prob_treatment, where="post",
             label="Choc = %s" % treatment_type)

plt.ylabel("Probabilté de survie $\hat{S}(t)$")
plt.xlabel("Temps en jours $t$")
plt.title("Comparaison avec ou sans choc")
plt.legend(loc="best")

In [ ]:
for treatment_type in (0, 1):
    mask_treat = hospit_full_hospit["CHIMIO"] == treatment_type
    time_treatment, survival_prob_treatment = kaplan_meier_estimator(
        hospit_full_hospit["Mort"][mask_treat],
        hospit_full_hospit["Nb_survie"][mask_treat])
    days=time_treatment.astype('timedelta64[D]')
    days/np.timedelta64(1,'D')
    plt.step(days, survival_prob_treatment, where="post",
             label="Chimiothérapie = %s" % treatment_type)

plt.ylabel("Probabilté de survie $\hat{S}(t)$")
plt.xlabel("Temps en jours $t$")
plt.title("Comparaison avec ou sans chimiothérapie")
plt.legend(loc="best")

In [ ]:
for treatment_type in (0, 1):
    mask_treat = hospit_full_hospit["HEMOD"] == treatment_type
    time_treatment, survival_prob_treatment = kaplan_meier_estimator(
        hospit_full_hospit["Mort"][mask_treat],
        hospit_full_hospit["Nb_survie"][mask_treat])
    days=time_treatment.astype('timedelta64[D]')
    days/np.timedelta64(1,'D')
    plt.step(days, survival_prob_treatment, where="post",
             label="Hémodialyse = %s" % treatment_type)

plt.ylabel("Probabilté de survie $\hat{S}(t)$")
plt.xlabel("Temps en jours $t$")
plt.title("Comparaison avec ou sans hémodialyse")
plt.legend(loc="best")

In [ ]:
# for treatment_type in (range(0,50,10)):
#     mask_treat =hospit_full_hospit["SEJ_NBJ"] <= treatment_type and hospit_full_hospit["SEJ_NBJ"]> treatment_type-10
#     time_treatment, survival_prob_treatment = kaplan_meier_estimator(
#         hospit_full_hospit["Mort"][mask_treat],
#         hospit_full_hospit["Nb_survie"][mask_treat])
#     days=time_treatment.astype('timedelta64[D]')
#     days/np.timedelta64(1,'D')
#     plt.step(days, survival_prob_treatment, where="post",
#              label="Durée de séjours à l'hopital <= %s" % treatment_type)

# plt.ylabel("Probabilté de survie $\hat{S}(t)$")
# plt.xlabel("Temps en jours $t$")
# plt.title("Comparaison avec la durée de séjour")
# plt.legend(loc="best")